In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from src.dataset2 import MIR1K2
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchaudio.models import HDemucs
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from collections import defaultdict
from src.constants import SAMPLE_RATE
from src.loss import bce
from src.utils import to_local_average_cents
from mir_eval.melody import (
    raw_pitch_accuracy,
    to_cent_voicing,
    raw_chroma_accuracy,
    overall_accuracy,
)

In [2]:
HOP_LENGTH = 20
SEQ_L = 2.56
BATCH_SIZE = 8
SAMPLE_RATE = 16000

train_dataset = MIR1K2("dataset", HOP_LENGTH, SEQ_L, groups= ["train"])
validation_dataset = MIR1K2('dataset', HOP_LENGTH, SEQ_L, groups= ["test"])

Loading 1 group of MIR1K2 at dataset


Loading group train: 100%|██████████| 800/800 [00:07<00:00, 101.93it/s]


Loading 1 group of MIR1K2 at dataset


Loading group test: 100%|██████████| 200/200 [00:01<00:00, 109.11it/s]


In [3]:
# import dataloader
data_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [4]:
def cycle(iterable):
    while True:
        for x in iterable:
            yield x

In [5]:
def conv_layer(input_channels, output_channels, kernel_size, stride):
    return nn.Sequential(
        nn.Conv1d(input_channels, output_channels, kernel_size, stride),
        nn.BatchNorm1d(output_channels),
        nn.ReLU(),
        nn.MaxPool1d(2, 2),
    )


class E2E_Demucs(nn.Module):
    def __init__(self):
        super(E2E_Demucs, self).__init__()
        self.hdemucs = HDemucs(sources=["vocals"], nfft=2048, depth=6, audio_channels=1)
        self.bigru = nn.GRU(1024, 256, 1, batch_first=True, bidirectional=True).cuda()
        self.conv = nn.Sequential(
            conv_layer(1, 512, 90, 1),
            conv_layer(512, 64, 12, 1),
            conv_layer(64, 128, 12, 1),
            conv_layer(128, 256, 12, 1),
        )
        self.fc = nn.Sequential(nn.Linear(512, 360), nn.Dropout(0.25), nn.Sigmoid())

    def forward(self, audio):
        x = self.hdemucs(audio)
        x = x.squeeze(1).view(x.size(0), -1, 320)
        B, S, T = x.shape
        x = self.conv(x.view(B * S, 1, T)).view(B, S, -1)
        x, _ = self.bigru(x)
        x = self.fc(x)
        return x

In [6]:
def evaluate2(dataset, model, hop_length, device, pitch_th=0.0):
    metrics = defaultdict(list)
    for data in dataset:
        audio = data['audio'].view(1,1,-1).to(device)
        pitch_label = data["pitch"].to(device)
        pitch_pred = model(audio).squeeze()

        loss = bce(pitch_pred, pitch_label)
        metrics["loss"].append(loss.item())

        cents_pred = to_local_average_cents(pitch_pred.cpu().numpy(), None, pitch_th)

        cents_label = to_local_average_cents(pitch_label.cpu().numpy(), None, pitch_th)

        freq_pred = np.array(
            [
                10 * (2 ** (cent_pred / 1200)) if cent_pred else 0
                for cent_pred in cents_pred
            ]
        )
        freq = np.array(
            [10 * (2 ** (cent / 1200)) if cent else 0 for cent in cents_label]
        )

        time_slice = np.array([i * hop_length / 1000 for i in range(len(cents_label))])
        ref_v, ref_c, est_v, est_c = to_cent_voicing(
            time_slice, freq, time_slice, freq_pred
        )

        rpa = raw_pitch_accuracy(ref_v, ref_c, est_v, est_c)
        rca = raw_chroma_accuracy(ref_v, ref_c, est_v, est_c)
        oa = overall_accuracy(ref_v, ref_c, est_v, est_c)
        metrics["RPA"].append(rpa)
        metrics["RCA"].append(rca)
        metrics["OA"].append(oa)
        # if rpa < 0.9:
        print(data["file"], ":\t", rpa, "\t", oa)

    return metrics


In [8]:
from src.loss import FL


log_dir = 'runs/E2E2'

learning_rate = 5e-4

validation_interval = len(data_loader) 
iterations = len(data_loader) * 100
learning_rate_decay_steps = len(data_loader) * 5
learning_rate_decay_rate = 0.98

os.makedirs(log_dir, exist_ok=True)
writer= SummaryWriter(log_dir)

model = E2E_Demucs().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=learning_rate_decay_steps, gamma=learning_rate_decay_rate)

loop = tqdm(range(1, iterations + 1))
RPA, RCA, OA, VFA, VR, it = 0, 0, 0, 0, 0, 0

for i, data in zip(loop, cycle(data_loader)):
    audio = data["audio"].unsqueeze(1).cuda()
    pitch_label = data["pitch"].cuda()
    pitch_pred = model(audio)
    
    loss = FL(pitch_pred, pitch_label, 6, 0)

    print(i, end="\t")
    print("loss_total:", loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()
    writer.add_scalar("loss/loss_pitch", loss.item(), global_step=i)

    if i % validation_interval == 0:
        model.eval()
        with torch.no_grad():
            metrics = evaluate2(validation_dataset, model, HOP_LENGTH, 'cuda')
            for key, value in metrics.items():
                writer.add_scalar(
                    "stage_pitch/" + key, np.mean(value), global_step=i
                )
            rpa = np.mean(metrics["RPA"])
            rca = np.mean(metrics["RCA"])
            oa = np.mean(metrics["OA"])
            if rpa >= RPA:
                RPA, RCA, OA, it = rpa, rca, oa, i
                with open(os.path.join(log_dir, "result.txt"), "a") as f:
                    f.write(str(i) + "\t")
                    f.write(str(RPA) + "\t")
                    f.write(str(RCA) + "\t")
                    f.write(str(OA) + "\t")
                torch.save(model, os.path.join(log_dir, f"model.pt"))
        model.train()

    if i - it > len(data_loader) * 10:
        break

  0%|          | 0/36400 [00:00<?, ?it/s]c:\Users\mond\miniconda3\envs\rmvpe\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


1	loss_total: 0.7013555765151978


  0%|          | 2/36400 [00:01<4:53:47,  2.06it/s]

2	loss_total: 0.6951654553413391


  0%|          | 3/36400 [00:01<3:41:49,  2.73it/s]

...

  1%|          | 363/36400 [01:24<2:19:32,  4.30it/s]

363	loss_total: 0.2111210972070694


  1%|          | 363/36400 [01:24<2:19:56,  4.29it/s]

364	loss_total: 0.21032653748989105


ValueError: Expected 3D tensor with dimensions (batch, channel, frames). Found: torch.Size([40960])